# ENGLISH BOOK DICTIONARY

* The aim of this project is to create an English Dictionary app that returns definitions of English words. When the application is started, it displays a menu as follows:

In [1]:
import os
import pickle

def add_word():
    word = input("Enter a word: ")
    meaning = input("Enter the meaning: ")
    words[word] = meaning
    save_words()

def find_meaning():
    word = input("Enter a word: ")
    if word in words:
        print(f"Meaning of '{word}': {words[word]}")
    else:
        print("Meaning not found.")

def update_word():
    word = input("Enter a word: ")
    if word in words:
        meaning = input("Enter the updated meaning: ")
        words[word] = meaning
        save_words()
    else:
        print("Word not found.")

def save_words():
    with open('words.txt', 'wb') as f:
        pickle.dump(words, f)

def load_words():
    if os.path.exists('words.txt'):
        with open('words.txt', 'rb') as f:
            return pickle.load(f)
    else:
        return {}

def main_menu():
    while True:
        print("\nMain Menu")
        print("1. Add a new word")
        print("2. Find the meaning")
        print("3. Update a word")
        print("4. Exit")
        choice = int(input("Enter Choice: "))
        if choice == 1:
            add_word()
        elif choice == 2:
            find_meaning()
        elif choice == 3:
            update_word()
        elif choice == 4:
            break
        else:
            print("Invalid choice. Please try again.")

words = load_words()
main_menu()


Main Menu
1. Add a new word
2. Find the meaning
3. Update a word
4. Exit

Main Menu
1. Add a new word
2. Find the meaning
3. Update a word
4. Exit

Main Menu
1. Add a new word
2. Find the meaning
3. Update a word
4. Exit
Meaning of 'gorgeous': splendid pr sumptuous in appearance

Main Menu
1. Add a new word
2. Find the meaning
3. Update a word
4. Exit


# Library Book Management System

* A Library Book Management System will be implemented with the front end GUI developed using Tkinter and the backend database implemented in SQLite. It is a single window tkinter GUI application which will implement CRUD( create, read, update, delete) operations on a ‘books’ database, in SQLite. The window will contain text widgets to accept / display book title, author, year of publication, ISBN (optional) and a large text area/ list box to display the records which match the search criteria

In [3]:
import tkinter as tk
from tkinter import messagebox
import sqlite3



def init_database():
    conn = sqlite3.connect('books.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS books
                 (title TEXT, author TEXT, year INTEGER, isbn TEXT, issued INTEGER)''')
    conn.commit()
    conn.close()



def search_books(title='', author='', isbn=''):
    conn = sqlite3.connect('books.db')
    c = conn.cursor()
    c.execute("SELECT * FROM books WHERE title LIKE ? OR author LIKE ? OR isbn LIKE ?",
              ('%' + title + '%', '%' + author + '%', '%' + isbn + '%'))
    result = c.fetchall()
    conn.close()

    # Clear the listbox before displaying new results
    listbox.delete(0, tk.END)
    for row in result:
        listbox.insert(tk.END, row)

def add_book(title, author, year, isbn):
    if title and author and year:
        conn = sqlite3.connect('books.db')
        c = conn.cursor()
        c.execute("INSERT INTO books VALUES (?, ?, ?, ?,0)", (title, author, year, isbn))
        conn.commit()
        conn.close()
        search_books(title, author, isbn)
        clear_entries()
    else:
        messagebox.showerror('Error', 'Title, author, and year are required.')

def issue_book(index):
    conn = sqlite3.connect('books.db')
    c = conn.cursor()
    c.execute("UPDATE books SET issued = 1 WHERE rowid = ?", (index + 1,))
    conn.commit()
    conn.close()
    search_books(listbox.get(index)[:3])

def delete_book(index):
    conn = sqlite3.connect('books.db')
    c = conn.cursor()
    c.execute("DELETE FROM books WHERE rowid = ?", (index + 1,))
    conn.commit()
    conn.close()
    search_books()

def update_book(index, title, author, year, isbn):
    conn = sqlite3.connect('books.db')
    c = conn.cursor()
    c.execute("UPDATE books SET title = ?, author = ?, year = ?, isbn = ? WHERE rowid = ?",
              (title, author, year, isbn, index + 1))
    conn.commit()
    conn.close()
    search_books()

def clear_entries():
    title_entry.delete(0, tk.END)
    author_entry.delete(0, tk.END)
    year_entry.delete(0, tk.END)
    isbn_entry.delete(0, tk.END)


init_database()

window = tk.Tk()
window.title('Library Book Management System')


main_frame = tk.Frame(window)
main_frame.grid(row=0, column=0, padx=10, pady=10)


title_label = tk.Label(main_frame, text='Book Title:')
title_label.grid(row=0, column=0, padx=5, pady=5, sticky='w')
title_entry = tk.Entry(main_frame, width=40)

title_entry.grid(row=0, column=1, padx=5, pady=5, sticky='w')

author_label = tk.Label(main_frame, text='Author:')
author_label.grid(row=1, column=0, padx=5, pady=5, sticky='w')
author_entry = tk.Entry(main_frame, width=40)
author_entry.grid(row=1, column=1, padx=5, pady=5, sticky='w')

year_label = tk.Label(main_frame, text='Year:')
year_label.grid(row=2, column=0, padx=5, pady=5, sticky='w')
year_entry = tk.Entry(main_frame, width=10)
year_entry.grid(row=2, column=1, padx=5, pady=5, sticky='w')

isbn_label = tk.Label(main_frame, text='ISBN:')
isbn_label.grid(row=3, column=0, padx=5, pady=5, sticky='w')
isbn_entry = tk.Entry(main_frame, width=20)
isbn_entry.grid(row=3, column=1, padx=5, pady=5, sticky='w')


search_button = tk.Button(main_frame, text='Search', command=lambda: search_books(title_entry.get(), author_entry.get(), isbn_entry.get()))
search_button.grid(row=4, column=0, padx=5, pady=5, sticky='w')
add_button = tk.Button(main_frame, text='Add', command=lambda: add_book(title_entry.get(), author_entry.get(), year_entry.get(), isbn_entry.get()))
add_button.grid(row=4, column=1, padx=5, pady=5)

scrollbar = tk.Scrollbar(window)
scrollbar.grid(row=1, column=3, sticky='ns')

listbox = tk.Listbox(window, height=12, width=50, yscrollcommand=scrollbar.set)
listbox.grid(row=1, column=2, padx=5, pady=5, sticky='nsew')
scrollbar.config(command=listbox.yview)

def listbox_click(event):
    selected_index = listbox.curselection()
    if len(selected_index) > 0:
        index = int(selected_index[0])
        selected_book = listbox.get(index)
        title_entry.delete(0, tk.END)
        title_entry.insert(tk.END, selected_book[0])
        author_entry.delete(0, tk.END)
        author_entry.insert(tk.END, selected_book[1])
        year_entry.delete(0, tk.END)
        year_entry.insert(tk.END, selected_book[2])
        isbn_entry.delete(0, tk.END)
        isbn_entry.insert(tk.END, selected_book[3])

listbox.bind('<<ListboxSelect>>', listbox_click)

def issue_books():
    selected_index = listbox.curselection()
    if len(selected_index) > 0:
        issue_book(int(selected_index[0]))

def delete_books():
    selected_index = listbox.curselection()
    if len(selected_index) > 0:
        delete_book(int(selected_index[0]))

def update_books():
    selected_index = listbox.curselection()
    if len(selected_index) > 0:
        title = title_entry.get()
        author = author_entry.get()
        year = year_entry.get()
        isbn = isbn_entry.get()
        update_book(int(selected_index[0]), title, author, year, isbn)

def reset_entries():
    clear_entries()
    listbox.delete(0, tk.END)


menu = tk.Menu(window)
window.config(menu=menu)

file_menu = tk.Menu(menu, tearoff=0)
menu.add_cascade(label="File", menu=file_menu)
file_menu.add_command(label="Add", command=lambda: add_book(title_entry.get(), author_entry.get(), year_entry.get(), isbn_entry.get()))
file_menu.add_command(label="Search", command=lambda: search_books(title_entry.get(), author_entry.get(), isbn_entry.get()))
file_menu.add_separator()
file_menu.add_command(label="Reset", command=reset_entries)

edit_menu = tk.Menu(menu, tearoff=0)
menu.add_cascade(label="Edit", menu=edit_menu)
edit_menu.add_command(label="Issue", command=issue_books)
edit_menu.add_command(label="Delete", command=delete_books)
edit_menu.add_command(label="Update", command=update_books)

init_database()


window.mainloop()